In [1]:
import pandas as pd
import numpy as np
import pickle
import random

In [2]:
df = pd.read_csv('../data/raw/test.csv.gz', compression='gzip')

C:\Users\marco\anaconda3\envs\capstone2env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,0000010489e3af074adeac69c53e555e,win8defender,1.1.15400.5,4.18.1810.5,1.281.501.0,0,7.0,0,NaN,53447.0,...,807.0,8554.0,1,NaN,0.0,0,0,0.0,0.0,7.0
1,00000176ac758d54827acd545b6315a5,win8defender,1.1.15400.4,4.18.1809.2,1.279.301.0,0,7.0,0,NaN,53447.0,...,554.0,33105.0,1,NaN,0.0,0,0,0.0,1.0,12.0
2,0000019dcefc128c2d4387c1273dae1d,win8defender,1.1.15300.6,4.18.1809.2,1.277.230.0,0,7.0,0,NaN,49480.0,...,556.0,63396.0,1,NaN,0.0,0,0,0.0,1.0,11.0
3,0000055553dc51b1295785415f1a224d,win8defender,1.1.15400.5,4.18.1810.5,1.281.664.0,0,7.0,0,NaN,42160.0,...,628.0,26320.0,1,0.0,0.0,0,0,0.0,0.0,10.0
4,00000574cefffeca83ec8adf9285b2bf,win8defender,1.1.15400.4,4.18.1809.2,1.279.236.0,0,7.0,0,NaN,53447.0,...,556.0,63269.0,1,NaN,0.0,0,0,0.0,1.0,3.0


In [4]:
df.shape

(7853253, 82)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853253 entries, 0 to 7853252
Data columns (total 82 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   MachineIdentifier                                  object 
 1   ProductName                                        object 
 2   EngineVersion                                      object 
 3   AppVersion                                         object 
 4   AvSigVersion                                       object 
 5   IsBeta                                             int64  
 6   RtpStateBitfield                                   float64
 7   IsSxsPassiveMode                                   int64  
 8   DefaultBrowsersIdentifier                          float64
 9   AVProductStatesIdentifier                          float64
 10  AVProductsInstalled                                float64
 11  AVProductsEnabled                                 

In [6]:
ID_sub = df.MachineIdentifier
df.drop('MachineIdentifier', inplace=True, axis=1)

In [7]:
to_drop = ['DefaultBrowsersIdentifier',
           'PuaMode',
           'Census_ProcessorClass',
           'Census_InternalBatteryType',
           'Census_IsFlightingInternal',
           'Census_ThresholdOptIn',
           'Census_IsWIMBootEnabled']

In [8]:
# Categorical Features Processing

df.loc[df.ProductName == 'scep', 'ProductName'] = np.nan
df.loc[df.Processor == 'arm64', 'Processor'] = np.nan
df.loc[df.OsVer == '6.1.0.0', 'OsVer'] = '6.1.1.0'
df.loc[df.OsVer == '10.0.3.0', 'OsVer'] = '10.0.0.0'
df.loc[df.OsVer == '10.0.0.2', 'OsVer'] = '10.0.0.0'
df.loc[df.OsVer == '10.0.7.0', 'OsVer'] = '10.0.0.0'
df.loc[df.OsVer == '10.0.1.0', 'OsVer'] = '10.0.0.0'
df.loc[df.SmartScreen == 'on', 'SmartScreen'] = 'On'
df.loc[df.SmartScreen == 'off', 'SmartScreen'] = 'Off'
df.loc[df.SmartScreen == '&#x01;', 'SmartScreen'] = '&#x0_'
df.loc[df.SmartScreen == '&#x02;', 'SmartScreen'] = '&#x0_'
df.loc[df.Census_DeviceFamily == 'Windows', 'Census_DeviceFamily'] = np.nan
df.loc[df.Census_PrimaryDiskTypeName == 'UNKNOWN', 'Census_PrimaryDiskTypeName'] = 'Unspecified'
df.loc[df.Census_ChassisTypeName == 'UNKNOWN', 'Census_ChassisTypeName'] = 'Unknown'
df.loc[df.Census_OSArchitecture == 'arm64', 'Census_OSArchitecture'] = np.nan

In [9]:
# Numerical Features Processing

df.IsProtected = df.IsProtected.map({np.nan:0, 0:1, 1:2})
limit = 16*10**6
df.loc[df['Census_PrimaryDiskTotalCapacity'] > limit, 'Census_PrimaryDiskTotalCapacity'] = np.nan
vol_limit = 2e6
df.loc[df['Census_SystemVolumeTotalCapacity'] > vol_limit, 'Census_SystemVolumeTotalCapacity'] = np.nan
df['Census_OSBranch_release_extracted'] = df.OsBuildLab.str.split('.').str.get(3)
to_drop = to_drop + ['IsBeta',
                     'AutoSampleOptIn',
                     'SMode',
                     'Census_IsPortableOperatingSystem',
                     'Census_IsFlightsDisabled',
                     'Census_IsVirtualDevice',
                     'OsBuildLab',
                     'Census_OSBuildNumber', 
                     'Census_OSBranch',
                     'UacLuaenable']

df.shape

(7853253, 82)

In [10]:
to_drop

['DefaultBrowsersIdentifier',
 'PuaMode',
 'Census_ProcessorClass',
 'Census_InternalBatteryType',
 'Census_IsFlightingInternal',
 'Census_ThresholdOptIn',
 'Census_IsWIMBootEnabled',
 'IsBeta',
 'AutoSampleOptIn',
 'SMode',
 'Census_IsPortableOperatingSystem',
 'Census_IsFlightsDisabled',
 'Census_IsVirtualDevice',
 'OsBuildLab',
 'Census_OSBuildNumber',
 'Census_OSBranch',
 'UacLuaenable']

In [11]:
df.drop(to_drop, axis=1, inplace=True)
df.shape

(7853253, 65)

In [12]:
df['EngineVersion_encoded'] = [item[2] for item in df.EngineVersion.str.split('.')]
df.loc[(df['EngineVersion_encoded'] != '15200') & (df['EngineVersion_encoded'] != '15100'),
       'EngineVersion_encoded'] = 'Other'

df['AppVersion_encoded'] = [item[1] for item in df.AppVersion.str.split('.')]
df.loc[df['AppVersion_encoded'] != '18', 'AppVersion_encoded'] = 'Other'

df['AvSigVersion_encoded'] = [item[1] for item in df.AvSigVersion.str.split('.')]
df.loc[(df['AvSigVersion_encoded'] != '275') & (df['AvSigVersion_encoded'] != '273'),
       'AvSigVersion_encoded'] = 'Other'

df['OsVer_encoded'] = np.where(df.OsVer == '10.0.0.0', 1, 0)

df['Census_OSVersion_encoded'] = [item[2] for item in df.Census_OSVersion.str.split('.')]
other_os = ~df['Census_OSVersion_encoded'].isin(['17134', 
                                                 '16299', 
                                                 '15063', 
                                                 '14393', 
                                                 '10586', 
                                                 '10240'])
df.loc[other_os, 'Census_OSVersion_encoded'] = 'Other'


to_drop = []
to_drop.append('EngineVersion')
to_drop.append('AppVersion')
to_drop.append('AvSigVersion')
to_drop.append('OsVer')
to_drop.append('Census_OSVersion')
df.drop(to_drop, axis=1, inplace=True)
df.shape

(7853253, 65)

In [13]:
#cat_col = df.select_dtypes('object').columns.tolist()
#high_card_col = [col for col in cat_col if df[col].nunique()>15]


other_chassis = ~df['Census_ChassisTypeName'].isin(['Notebook', 
                                                    'Desktop', 
                                                    'Laptop'])
df.loc[other_chassis, 'Census_ChassisTypeName'] = 'Other'

other_os_edition = ~df['Census_OSEdition'].isin(['Core',
                                                'Professional',
                                                'CoreSingleLanguage'])
df.loc[other_os_edition, 'Census_OSEdition'] = 'Other'

other_os_sku = ~df['Census_OSSkuName'].isin(['CORE',
                                            'PROFESSIONAL',
                                            'CORE_SINGLELANGUAGE'])
df.loc[other_os_sku, 'Census_OSSkuName'] = 'Other'

df.loc[df['Census_OSBranch_release_extracted'].str.contains('rs1', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'rs1_release'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('rs2', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'rs2_release'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('rs3', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'rs3_release'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('rs4', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'rs4_release'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('rs5', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'rs5_release'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('win', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'win'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('th2', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'th'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('th1', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'th'
df.loc[df['Census_OSBranch_release_extracted'].str.contains('rs_pre', na=False, regex=False), 'Census_OSBranch_release_extracted'] = 'rs_prerelease'

other_os_branch = ~df['Census_OSBranch_release_extracted'].isin(['rs4_release',
                                                                 'rs3_release',
                                                                 'rs2_release',
                                                                 'rs1_release',
                                                                 'th'
                                                                ])
df.loc[other_os_branch, 'Census_OSBranch_release_extracted'] = 'Other'

df.shape

(7853253, 65)

In [14]:
with open('../data/processed/X_train', 'rb') as f:
    X_train = pickle.load(f)

In [15]:
X_test_sub = df[X_train.columns]
X_test_sub.head()

,Census_ProcessorCoreCount,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,IsSxsPassiveMode,HasTpm,...,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Wdft_RegionIdentifier
0,4.0,488386.0,123179.0,8192.0,15.5,1920.0,1080.0,8.0,0,1,...,2689.0,30661.0,5.0,3063.0,1387,37.0,158,807.0,8554.0,7.0
1,4.0,1907729.0,1882352.0,8192.0,15.5,1366.0,768.0,0.0,0,1,...,2206.0,242491.0,5.0,3410.0,611,7.0,30,554.0,33105.0,12.0
2,2.0,29820.0,28678.0,4096.0,13.9,1280.0,720.0,0.0,0,1,...,585.0,189533.0,5.0,2097.0,2189,35.0,148,556.0,63396.0,11.0
3,2.0,476940.0,439345.0,4096.0,14.0,1366.0,768.0,0.0,0,1,...,2668.0,171228.0,5.0,1985.0,371,26.0,119,628.0,26320.0,10.0
4,4.0,476940.0,461506.0,2048.0,15.5,1366.0,768.0,0.0,0,1,...,585.0,189538.0,5.0,3394.0,371,29.0,125,556.0,63269.0,3.0


In [16]:
with open ('../data/processed/X_test_kaggle_submission.pickle', 'wb') as handle:
    pickle.dump(X_test_sub, handle)

In [17]:
with open ('../data/processed/X_test_kaggle_submission_MachineIdintifier.pickle', 'wb') as handle:
    pickle.dump(ID_sub, handle)